# n-step Algorithm

This notebook attempts to compare a correct n-step method with a method using the sum of TD errors. The difference between the two methods is that the value estimates are changing online within an episode for the first method. In theory, the methods are very similar but slightly different.

Both methods used to solve a simple problem: the Random Walk MRP. The objective is to approximate the true value function correctly (called _prediction problem_ in the book). An experiment is conducted using a statistical test to find out whether the performance of the correct method is significantly better.

The basic design is very similar to other notebooks.

Some implementation difficulty that I face with Julia is that arrays start with 1 as opposed to 0 in other languages. This is not a problem per se - for the Reinforcement Learning algorithms in the book it is even advantageous - but it is hard to let go of the internal 0-based bias.

In [17]:
using Statistics
using HypothesisTests

┌ Info: Precompiling HypothesisTests [09f84164-cd44-5f33-b23f-e6b0d136a0d5]
└ @ Base loading.jl:1260


In [2]:
gamma = 1.
alpha = .2
n = 10
episodes = 10

10

In [3]:
mutable struct Walk
    agent::Int64
    num_states::Int64
end

reset_state(w::Walk) = w.agent = w.num_states ÷ 2

function act(w::Walk)
    if rand() < .5
        w.agent += 1
    else
        w.agent -= 1
    end
    
    reward = 0
    episode_end = false
    if w.agent >= num_states
        reward = 1
        episode_end = true
    elseif w.agent <= 1
        reward = -1
        episode_end = true
    end
    reward, episode_end
end

act (generic function with 1 method)

In [4]:
"""
Applies nstep TD on the random walk MRP. Optionally uses unchanging value functions. Returns the value table.
"""
function nstep(w::Walk, n::Int64, episodes::Int64; gamma=gamma, alpha=alpha, changing=true)
    values = zeros(w.num_states)
    values_old = zeros(w.num_states)
    for episode in 1:episodes
        reset_state(w)
        step = 0
        episode_end = false
        T = typemax(Int64)
        tau = 0
        rewards = []
        states = []
        while tau != T - 1
            step += 1
            if step <= T
                reward, episode_end = act(w)
                push!(rewards, reward)
                push!(states, w.agent)
                if episode_end
                    T = step + 1
                    reset_state(w)
                end
            end
            tau = step - n + 1
            if tau >= 1
                return_step = min(tau + n, T)
                gammas = cumprod(fill(gamma, return_step - tau))
                G = sum(rewards[tau:return_step - 1] .* gammas)
                G += tau + n < T ? gamma ^ n * values_old[states[tau + n - 1]] : 0
                values[states[tau]] += alpha * (G - values_old[states[tau]])
                if changing
                    values_old[states[tau]] = values[states[tau]]
                end
            end
        end
        if !changing
            values_old = values
        end
    end
    
    values
end

nstep

In [5]:
num_states = 19 + 2
agent = num_states ÷ 2
walk = Walk(agent, num_states)

Walk(10, 21)

In [6]:
nstep(walk, n, episodes)

21-element Array{Float64,1}:
 -0.5904
 -0.8528203308174283
 -0.799320860656892
 -0.694034622393137
 -0.6916667051838917
 -0.5936317199939259
 -0.38876809556067105
  0.19009150362652638
  0.19947120955522937
  0.21485636094652014
  0.21707796472633872
  0.5528741220452527
  0.5735165318660564
  0.5610515667203062
  0.6203529386825122
  0.721683548661922
  0.7703373853255335
  0.8357964412978435
  0.9078156036703668
  0.9158146268284806
  0.7378560000000001

In [7]:
nstep(walk, n, episodes, changing=false)

21-element Array{Float64,1}:
 -0.7378560000000001
 -0.7691466057621505
 -0.8344234718173119
 -0.7755481156423978
 -0.4870825749867077
 -0.3017102574585919
 -0.2511718167024796
 -0.26341645358740695
 -0.2426155344585718
 -0.01830383556051879
  0.03978171923319583
  0.03719922235414681
  0.08387314268080258
  0.16609085255404343
  0.3272412831430848
  0.3846884387502305
  0.4318437244954966
  0.5953649116779176
  0.7424991367578111
  0.7902848
  0.5904

In [8]:
half = walk.num_states ÷ 2
true_values = collect(-half:half) ./ half

21-element Array{Float64,1}:
 -1.0
 -0.9
 -0.8
 -0.7
 -0.6
 -0.5
 -0.4
 -0.3
 -0.2
 -0.1
  0.0
  0.1
  0.2
  0.3
  0.4
  0.5
  0.6
  0.7
  0.8
  0.9
  1.0

In [62]:
"""
Runs the experiment multiple times.
"""
function run_experiment(n; changing=true)
    runs = 100
    metrics = zeros(runs)
    for run in 1:runs
        values = nstep(walk, n, episodes, changing=changing)
        metric = sqrt(mean((values - true_values) .^ 2))
        metrics[run] = metric
    end
    metrics
end

run_experiment (generic function with 1 method)

In [57]:
metrics_nstep = run_experiment(4);

In [58]:
metrics_tdsum = run_experiment(4, changing=false);

In [68]:
EqualVarianceTTest(metrics_nstep, metrics_tdsum)

Two sample t-test (equal variance)
----------------------------------
Population details:
    parameter of interest:   Mean difference
    value under h_0:         0
    point estimate:          -0.00914756824809465
    95% confidence interval: (-0.0178, -0.0005)

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0391

Details:
    number of observations:   [100,100]
    t-statistic:              -2.076487997079754
    degrees of freedom:       198
    empirical standard error: 0.004405307548591291


In order to compare the two methods, they are evaluated using the root mean squared error to the true value over 100 runs. Then a statistical test can be used with two hypotheses and a significance level $\alpha=0.05$:

* $H_0$: the evaluation metrics for both methods come from the same distribution (i.e. both methods are similar in performance)
* $H_A$: the evaluation metrics come from different distributions

The above table displays the result of a two sample t-test, which concerns the mean $\mu$ as distribution parameter. The result of the test is that $H_0$ can be rejected because the p-value is lower than $\alpha$. This means that the performance of the correct n-step method is statistically significantly better than the performance of the sum of TD errors method.

In order to make sure that the test is not skewed due to the non-robust assumptions, a Mann-Whitney U test is performed below.

In [70]:
MannWhitneyUTest(metrics_nstep, metrics_tdsum)

Approximate Mann-Whitney U test
-------------------------------
Population details:
    parameter of interest:   Location parameter (pseudomedian)
    value under h_0:         0
    point estimate:          -0.006745996303989921

Test summary:
    outcome with 95% confidence: reject h_0
    two-sided p-value:           0.0377

Details:
    number of observations in each group: [100, 100]
    Mann-Whitney-U statistic:             4149.0
    rank sums:                            [9199.0, 10901.0]
    adjustment for ties:                  0.0
    normal approximation (μ, σ):          (-851.0, 409.2676385936225)
